#Multi-Agent Order Management System with MongoDB

This notebook implements a multi-agent system for managing product orders, inventory, and deliveries using:

smolagents for agent management
MongoDB for data persistence
DeepSeek Chat as the LLM model

##Setting Up MongoDB Atlas

Create a free MongoDB Atlas account at https://www.mongodb.com/cloud/atlas/register

- Create a new cluster (free tier is sufficient)
- Configure network access by adding your IP address
- Create a database user with read/write permissions
- Get your connection string from Atlas UI (Click "Connect" > "Connect your application")
- Replace <password> in the connection string with your database user's password
- Enable network access from your IP address in the Network Access settings

###Security Considerations

When working with MongoDB Atlas:

- Never commit connection strings with credentials to version control
- Use environment variables or secure secret management
- Restrict database user permissions to only what's needed
- Enable IP allowlist in Atlas Network Access settings

##Setup

First, let's install required dependencies:

In [10]:
!pip install smolagents pymongo litellm


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
!pip install 'smolagents[litellm]'


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!pip install "pymongo[srv]"


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
from smolagents.agents import ToolCallingAgent
from smolagents import tool, LiteLLMModel, CodeAgent
from pymongo import MongoClient
from datetime import datetime
from typing import List, Dict, Optional
import os

# Initialize LLM model
MODEL_ID = "gemini/gemini-2.0-flash"
MONGODB_URI = os.getenv('MONGO_URI')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# MongoDB Connection manager
mongoclient = MongoClient(MONGODB_URI, tls=True, tlsAllowInvalidCertificates=True, appname="devrel.showcase.multi-smolagents")
db = mongoclient.warehouse

In [5]:
from pymongo import MongoClient

try:
    client = MongoClient(MONGODB_URI)
    db = client.test
    print("Connected to MongoDB")
except Exception as e:
    print(f"Connection error: {e}")

Connected to MongoDB


In [6]:
@tool
def check_stock(product_id: str) -> Dict:
    """Query product stock level.

    Args:
        product_id: Product identifier

    Returns:
        Dict containing product details and quantity
    """
    return db.products.find_one({"_id": product_id})

@tool
def update_stock(product_id: str, quantity: int) -> bool:
    """Update product stock quantity.

    Args:
        product_id: Product identifier
        quantity: Amount to decrease from stock

    Returns:
        bool: Success status
    """
    result = db.products.update_one(
        {"_id": product_id},
        {"$inc": {"quantity": -quantity}}
    )
    return result.modified_count > 0

In [7]:
from uuid import uuid4

@tool
def generate_order_id() -> str:
    """Generate a unique random hexadecimal order ID.

    Returns:
        str: Unique 24-character hex string
    """
    return uuid4().hex[:24]

In [8]:
@tool
def create_order( products: any, address: str) -> str:
    """Create new order for all provided products.

    Args:
        products: List of products with quantities
        address: Delivery address

    Returns:
        str: Order ID message (Order identifier, it must be a 12-byte input or a 24-character hex string)
    """
    order = {
        "products": products,
        "status": "pending",
        "delivery_address": address,
        "created_at": datetime.now()
    }
    result = db.orders.insert_one(order)
    return f"Successfully ordered : {str(result.inserted_id)}"

In [9]:
from bson.objectid import ObjectId
@tool
def update_delivery_status(order_id: str, status: str) -> bool:
    """Update order delivery status to in_transit once a pending order is provided

    Args:
        order_id: Order identifier it must be a 12-byte input or a 24-character hex string
        status: New delivery status (e.g., "in_transit" or "delivered")

    Returns:
        bool: Success status
    """
    if status not in ["pending", "in_transit", "delivered", "cancelled"]:
        raise ValueError("Invalid delivery status")

    result = db.orders.update_one(
        {"_id":  ObjectId(order_id), "status": "pending"},
        {"$set": {"status": status}}
    )
    return result.modified_count > 0

In [11]:
class OrderManagementSystem:
    """Multi-agent order management system"""
    def __init__(self, model_id: str = MODEL_ID):
        self.model = LiteLLMModel(model_id=model_id, api_key=GOOGLE_API_KEY)

        # Create agents
        self.inventory_agent = ToolCallingAgent(
            tools=[check_stock, update_stock],
            model=self.model,
            name="inventory",
            description="Manages product inventory"
        )

        self.order_agent = ToolCallingAgent(
            tools=[create_order, generate_order_id],
            model=self.model,
            name="orders",
            description="Handles order creation",
        )

        self.delivery_agent = ToolCallingAgent(
            tools=[update_delivery_status],
            model=self.model,
            name="delivery",
            description="Manages delivery status"
        )

        self.id_generator = ToolCallingAgent(
            tools=[generate_order_id],
            model=self.model,
            name="id_generator",
            description="Generates unique order IDs"
        )

        # Create managed agents
        self.managed_agents = [self.inventory_agent, self.order_agent, self.delivery_agent, self.id_generator]

        # Create manager agent
        self.manager = CodeAgent(
            tools=[],
            # system_prompt="""For each order:
            # 1. Create the order document
            # 2. Update the inventory
            # 3. Set delivery status to in_transit

            # Use relevant agents:  {{managed_agents_descriptions}}  and you can use {{authorized_imports}}
            # """,
            model=self.model,
            managed_agents=self.managed_agents,
            additional_authorized_imports=["time", "json", "uuid"]
        )

    def process_order(self, orders: List[Dict]) -> str:
        """Process a set of orders.

        Args:
            orders: List of orders each has address and products

        Returns:
            str: Processing result
        """
        return self.manager.run(
            f"Process the following  {orders} as well as substract the ordered items from inventory."
            f"to be delivered to relevant addresses"
        )

In [ ]:
def add_sample_products():
    db.products.delete_many({})
    sample_products = [
        {"_id": "prod1", "name": "Laptop", "price": 999.99, "quantity": 10},
        {"_id": "prod2", "name": "Smartphone", "price": 599.99, "quantity": 15},
        {"_id": "prod3", "name": "Headphones", "price": 99.99, "quantity": 30}
    ]

    db.products.insert_many(sample_products)
    print("Sample products added successfully!")

# Uncomment to add sample products
# add_sample_products()

Sample products added successfully!


In [14]:
# Initialize system
system = OrderManagementSystem()

# Create test orders
test_orders = [
    {
        "products": [
            {"product_id": "prod1", "quantity": 2},
            {"product_id": "prod2", "quantity": 1}
        ],
        "address": "123 Main St"
    },
    {
        "products": [
            {"product_id": "prod3", "quantity": 3}
        ],
        "address": "456 Elm St"
    }
]

# Process order
result = system.process_order(
    orders=test_orders
)

print("Orders processing result:", result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Process the following  [{'products': [{'product_id': 'prod1', 'quantity': 2}, {'product_id': 'prod2',           │
│ 'quantity': 1}], 'address': '123 Main St'}, {'products': [{'product_id': 'prod3', 'quantity': 3}], 'address':   │
│ '456 Elm St'}] as well as substract the ordered items from inventory.to be delivered to relevant addresses      │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  orders_data = [{'products': [{'product_id': 'prod1', 'quantity': 2}, {'product_id': 'prod2', 'quantity': 1}],    
  'address': '123 Main St'}, {'products': [{'product_id': 'prod3', 'quantity': 3}], 'address': '456 Elm St'}]      
                                                                                                                   
  for order in orders_data:                                                                                        
      print(order)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{'products': [{'product_id': 'prod1', 'quantity': 2}, {'product_id': 'prod2', 'quantity': 1}], 'address': '123 Main
St'}
{'products': [{'product_id': 'prod3', 'quantity': 3}], 'address': '456 Elm St'}

Out: None

[Step 0: Duration 8.17 seconds| Input tokens: 2,384 | Output tokens: 150]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Inventory updated and deliveries scheduled.")                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Inventory updated and deliveries scheduled.

[Step 5: Duration 0.93 seconds| Input tokens: 22,275 | Output tokens: 1,571]

Orders processing result: Inventory updated and deliveries scheduled.
